<a href="https://colab.research.google.com/github/alirouzbayani1/pericardiocentesis-cardiac-wall-detection/blob/main/wall_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://github.com/tensorpack/tensorpack/blob/master/examples/HED/hed.py
!wget http://models.tensorpack.com/HED/HED_reproduced.npz

In [ ]:
import cv2
import os

In [ ]:
def video_to_frames(video_path, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    vidcap = cv2.VideoCapture(video_path)
    video_name = os.path.basename(os.path.splitext(video_path)[0])
    success, image = vidcap.read()
    count = 0
    while success:
        cv2.imwrite(f"{output_path}{video_name}_frame{count}.jpg", image)     # save frame as JPEG file      
        success, image = vidcap.read()
        count += 1
def videos_to_frames(videos_path, output_path):
    for filename in os.listdir(videos_path):
        video_to_frames(videos_path + filename, output_path)


In [ ]:
videos_to_frames('./sample_videos/', './sample_frames/')

In [ ]:
!./hed.py --load HED_reproduced.npz --run ./sample_frames/a_frame0.jpg